In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import pandas as pd

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import ParameterGrid
from google.colab import drive

drive.mount('/content/drive')

# 재현 가능성을 위한 시드 고정
RANDOM_SEED = 18
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Dataloader 시드 고정 (아직 사용 x)

generator = torch.Generator()
generator.manual_seed(RANDOM_SEED)

def worker_init_fn(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# DataLoader(worker_init_fn=worker_init_fn)

# 기타 경로 설정 : 저장되는 데이터 이름 등 (model, method 에 맞게 설정)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# 데이터 경로 설정
train_csv_path = "/content/drive/MyDrive/Colab Notebooks/contest/samsung/train.csv"
test_csv_path = "/content/drive/MyDrive/Colab Notebooks/contest/samsung/test.csv"
submission_csv_path = '/content/drive/MyDrive/Colab Notebooks/contest/samsung/sample_submission.csv'

In [20]:
# MLP - 은닉층 5개
# 현재 최고성능 모델 하이퍼파라미터(GS) : {"batch_size": 32, "hidden_sizes": [32, 64, 64, 32, 16], "learning_rate": 0.001}
# Epoch : 93
# 제출 성능 : 0.752
class MLP5Hidden(nn.Module):
    def __init__(self, input_size=11, hidden_sizes=[32, 64, 64, 32, 16], output_size=1):
        super(MLP5Hidden, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.fc5 = nn.Linear(hidden_sizes[3], hidden_sizes[4])
        self.fc6 = nn.Linear(hidden_sizes[4], output_size)
        self.elu = nn.ELU()

    # dropout
    def forward(self, x):
        x = self.elu(self.fc1(x))
        x = self.elu(self.fc2(x))
        x = self.elu(self.fc3(x))
        x = self.elu(self.fc4(x))
        x = self.elu(self.fc5(x))
        x = self.fc6(x)
        return x


In [15]:
# 훈련 데이터를 훈련 + 검증 데이터로 나누고 train_dataset, val_dataset 을 반환하는 함수:
train_data = pd.read_csv(train_csv_path)

X = torch.tensor(train_data.iloc[:,1:-1].values, dtype=torch.float32)
y = torch.tensor(train_data.iloc[:,-1].values, dtype = torch.float32).view(-1,1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

In [26]:
batch_size = 32
learning_rate = 0.001

model = MLP5Hidden()
model.cuda()
criterion = nn.MSELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=worker_init_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []

# 검증 손실이 정체되면 학습률을 줄임
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# early stop
best_val_loss = float('inf')
patience = 10
trigger_times = 0

# epochs
epochs = 93
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss / len(train_loader)
    train_losses.append(train_loss)


    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()

    val_loss = val_loss / len(val_loader)
    val_losses.append(val_loss)

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
    scheduler.step(val_loss)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/93, Train Loss: 351.1599, Validation Loss: 7.6320
Epoch 2/93, Train Loss: 4.0716, Validation Loss: 3.6898
Epoch 3/93, Train Loss: 3.1899, Validation Loss: 4.0620
Epoch 4/93, Train Loss: 3.0790, Validation Loss: 3.5019
Epoch 5/93, Train Loss: 3.1798, Validation Loss: 3.4256
Epoch 6/93, Train Loss: 3.1374, Validation Loss: 3.3070
Epoch 7/93, Train Loss: 3.1138, Validation Loss: 3.3144
Epoch 8/93, Train Loss: 3.1415, Validation Loss: 3.5998
Epoch 9/93, Train Loss: 3.1019, Validation Loss: 3.3154
Epoch 10/93, Train Loss: 3.0714, Validation Loss: 3.4794
Epoch 11/93, Train Loss: 3.0060, Validation Loss: 6.1307
Epoch 12/93, Train Loss: 3.1176, Validation Loss: 3.3682
Epoch 13/93, Train Loss: 2.7923, Validation Loss: 3.2853
Epoch 14/93, Train Loss: 2.7916, Validation Loss: 3.2657
Epoch 15/93, Train Loss: 2.7954, Validation Loss: 3.5817
Epoch 16/93, Train Loss: 2.8039, Validation Loss: 3.2657
Epoch 17/93, Train Loss: 2.8060, Validation Loss: 3.2800
Epoch 18/93, Train Loss: 2.7984, Valid

In [27]:
X_test = pd.read_csv(test_csv_path).iloc[:,1:]
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
model.eval()
test_pred = []
with torch.no_grad():
    for batch in test_loader:
        batch_X = batch[0].cuda()
        outputs = model(batch_X)
        test_pred.extend(outputs.cpu().numpy())  # 예측값을 CPU로 이동하여 리스트에 추가

test_pred = np.array(test_pred).flatten()
submission_df = pd.read_csv(submission_csv_path)
submission_df['y'] = test_pred  # 예측 결과를 y 컬럼에 추가

# 지정된 경로에 CSV 파일로 저장
submission_df.to_csv("/content/drive/MyDrive/Colab Notebooks/contest/samsung/result/new_MLP5_PReLU.csv", index=False)

In [28]:
submission_df.describe()

,y
count,4986.000000
mean,84.926102
std,3.708393
min,81.510361
25%,82.285774
50%,83.134861
75%,86.257811
max,95.057632


In [29]:
test_pred = []
with torch.no_grad():
    for batch in test_loader:
        batch_X = batch[0].cuda()
        outputs = model(batch_X)
        test_pred.extend(outputs.cpu().numpy())

last_pred = np.array(test_pred)
threshold = np.percentile(last_pred, 90)
print(f"Top 10% threshold: {threshold:.4f}")

Top 10% threshold: 91.7425
